In [175]:
import re
import pandas as pd

In [96]:
with open('./data/Lexis Cases txt/P1.txt') as file:
    doc_count = 0
    conclusion_count = 0
    lines = file.readlines()
#     doc_info = dict()
    case_no = []
    registries = []
    paras = []
    curr_case = ''
    for line in lines:
        match_case = re.search(r'\[[0-9]{4}\] [[A-Z|\.]+ No\. [0-9]+', line)
        match_reg = re.match(r'Registry: ?([A-Za-z|\s]+)', line)
        match_paras = re.match(r'\(([0-9]+) paras\.?\)', line)
        if match_reg:
            registries.append(match_reg.group(1).strip())
        if match_case:
            case_no.append(match_case.group(0))
        if match_paras:
            paras.append(match_paras.group(1))
        if line == 'End of Document\n':
            doc_count+=1
        if re.match('conclusions?', line, re.IGNORECASE) and len(line.split()) < 10:
            print(line)
            
            
            

Conclusion

Conclusion re Future Care

CONCLUSION - DAMAGE AWARD

Conclusion re Future Care

CONCLUSION - DAMAGE AWARD

Conclusion

Conclusion

Conclusion on Contributory Negligence

CONCLUSION ON MS. HOWELL'S ONGOING DIFFICULTIES

CONCLUSION

Conclusion

Conclusion

Conclusion

Conclusion

Conclusion on Contributory Negligence

Conclusion on Ms. Howell's Ongoing Difficulties

Conclusion

Conclusion

Conclusion

Conclusion

Conclusion

Conclusion

CONCLUSIONS ON LIABILITY

Conclusion re Foreseeable Risks of Harm

CONCLUSION

CONCLUSIONS ON LIABILITY

Conclusion on Length of Time Patient was Unmonitored

Conclusion re Foreseeable Risks of Harm

CONCLUSION

Conclusion

Conclusion

Conclusion regarding Shannon Kemp's November 2006 admission to VGH

Conclusion regarding the Plaintiff's claims as against VCHA

Conclusion:

CONCLUSION:

CONCLUSION

CONCLUSION

CONCLUSION RE: CLAIMS IN NEGLIGENCE

Conclusion

CONCLUSION

Conclusion on presence of degenerative disc

CONCLUSION:

Conclusion on 

In [231]:
with open('./data/Lexis Cases txt/P1.txt') as file:
    lines = file.read()

In [228]:
def parse_file(path):
    '''given file path, extract relevant info for each document in file 
    Return list of dictionary of document features per document '''
    with open(path) as file:
        docs = file.read().split('End of Document\n')[:-1]
    
    docs_feats = []
    for i in range(len(docs)):

        doc_dict = dict()
        case_no = re.search(r'\/P([0-9]+)', path).group(1)
        doc = docs[i]
        lines = doc.split('\n')
        case_name = lines[0]
        registry = re.search(r'Registry:? ?([A-Za-z|\s|0-9|\.]+)', doc)
        no_paras = re.search(r'\(([0-9|,]+) paras?\.?\)', doc)
        doc_dict['case number'] = '%s of %s'%(i+1+((int(case_no)-1)*50), case_no)
        doc_dict['case name'] = case_name
        if registry:
            doc_dict['registry'] = registry.group(1).strip()
        else:
            doc_dict['registry'] = None
            
        if no_paras:
            doc_dict['decision length'] = no_paras.group(1)
        else:
            doc_dict['decision length'] = None
        doc_dict['contributory negligence raised'] = False
        
        damages_headers = set()
        conclusions = set()
        for line in lines:
            
            if re.match('contributory negligence', line, re.IGNORECASE): #this isnt working...
                doc_dict['contributory negligence raised'] = True
                
            concl_match = re.match('conclusions?', line, re.IGNORECASE)
            if concl_match and len(line.split()) < 10:
                conclusions.add(line)
            damages = re.match('damages', line, re.IGNORECASE)
            if damages and len(line.split()) < 10:
                damages_headers.add(line)
        doc_dict['damages_headers'] = damages_headers
        doc_dict['conclusion_headers'] = conclusions
        if damages_headers != set():
            for damages in damages_headers:
                text = doc.split(damages)[-1]
                damages_mentioned = re.findall(r'\$[0-9|,]+', text)
                if len(damages_mentioned) > 0:
                    doc_dict['$$ under header <%s>'%damages] = damages_mentioned
                    
        if conclusions != set():
            for conc in conclusions:
                text = doc.split(conc)[-1]
                damages_mentioned = re.findall(r'\$[0-9|,]+', text)
                if len(damages_mentioned) > 0:
                    doc_dict['$$ under header <%s>'%conc] = damages_mentioned            
        docs_feats.append(doc_dict)
#     df = pd.DataFrame(docs_feats)
    return docs_feats

In [229]:
docs_feats = parse_file('./data/Lexis Cases txt/P2.txt')
docs_feats

[{'case number': '51 of 2',
  'case name': 'Morrow v. Outerbridge, 2009 CHFL para. 15,554',
  'registry': None,
  'decision length': None,
  'contributory negligence raised': False,
  'damages_headers': set(),
  'conclusion_headers': set()},
 {'case number': '52 of 2',
  'case name': "Nelson v. Provincial Health Services Authority (c.o.b. British Columbia Women's Hospital and Health Centre), [2015] B.C.J. No. 2316",
  'registry': 'Victoria',
  'decision length': '166',
  'contributory negligence raised': False,
  'damages_headers': {'Damages'},
  'conclusion_headers': set(),
  '$$ under header <Damages>': ['$17,415']},
 {'case number': '53 of 2',
  'case name': 'Ruchelski v. Moore, [2013] B.C.J. No. 561',
  'registry': 'Vernon',
  'decision length': '110',
  'contributory negligence raised': False,
  'damages_headers': set(),
  'conclusion_headers': set()},
 {'case number': '54 of 2',
  'case name': 'Steinebach (Litigation guardian of) v. Fraser Health Authority (c.o.b. Surrey Memorial

In [232]:
docs = lines.split('End of Document\n')


In [83]:
seq = re.compile(r"conclusions?:?.*", re.MULTILINE|re.IGNORECASE)
for match in seq.finditer(docs[1]):
    print(match.group(0))

CONCLUSIONS
Conclusion re Future Care
CONCLUSION - DAMAGE AWARD
CONCLUSIONS
conclusions are these.
conclusion of the forceps attempt.
conclusion of the forceps attempt.
conclusion that it is more likely than not the conduct of the defendant was a cause in fact of the result. A mere possibility of such causation is not enough; and when the matter remains one of pure speculation or conjecture, or the probabilities are at best evenly balanced, it becomes the duty of the court to direct a verdict for the defendant.
conclusion in Cherry v. Borsman, as described above, may no longer apply. This enlarged position effectively subsumes Dr. Johnston's original position. However, because of the manner in which the case was presented and argued in the trial, with further written submissions after judgment was reserved, it seems most convenient and responsive to the weight and direction of counsels' submissions to address first the matter of informed consent and the fetus, and then the larger quest